In [ ]:
import os
import sys
from google.colab import drive 
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/ICBF/data


# **Desnutrición por municipio**

####**Descripción:**
Genera una tabla de datos con la cantidad de beneficiarios total y en cada condición por municipio, y per cápita, según datos poblacionales del Censo Nacional de Población y Vivienda 2018. Además calcula:
- de los beneficiarios en cada municipio, la fracción que sufrió desnutrición al menos una vez;
- de los beneficiarios que sufrieron desnutrición, la fracción que se recuperó;
- de los beneficiarios que se recuperaron, la fracción que reincidió en desnutrición.
También crea dos tablas de datos con muestras de las tomas nutricionales, incluyendo un máximo de 500 ó de 50 beneficiarios por cada municipio para su prueba en la aplicación.
 
####**Requerimientos:**

- Tabla de datos `clean_data/sociodemo.parquet` con los datos sociodemográficos y las etiquetas de cada beneficiario.
- Tabla de relación entre los identificadores IdBeneficiario y las etiquetas `clean_data/IdBeneficiario_labels.parquet`.
- Tabla de datos nutricionales limpia `clean_data/tomas_nutricionales.parquet`.
- Tabla de datos `auxiliary_data/municipio_poblacion_sexo_edad_pib.csv` con datos demográficos y económicos por municipio procesado a partir de la [*Serie municipal de población por área, sexo y edad, para el periodo 2018 -2026*](https://www.dane.gov.co/index.php/estadisticas-por-tema/demografia-y-poblacion/proyecciones-de-poblacion) con las proyecciones poblacionales elaboradas por el DANE a partir de los datos del Censo Nacional de Población y Vivienda 2018. Contiene la población proyectada de niños y niñas de 0 a 5 años, discriminada por sexo, edad y año y el PIB para cada municipio.
 
####**Salidas:**

- Tabla de datos `app/incidencia_mpio.parquet` con la cantidad de beneficiarios en cada condición y las fracciones relevantes por municipio.
- Tabla de datos `app/tomas_max_500.parquet` con tomas nutricionales y algunos datos socioeconómicos, para una muestra de máximo 500 beneficiarios por municipio.
- Tabla de datos `app/tomas_max_50.parquet` con tomas nutricionales y algunos datos socioeconómicos, para una muestra de máximo 50 beneficiarios por municipio. 

In [ ]:
# Librerias relevantes
import time
import pandas as pd
import numpy as np

### **Lectura de archivos**

In [ ]:
t0 = time.time()
# Lee la tabla de datos sociodemográficos 
soc = pd.read_parquet('clean_data/sociodemo.parquet')
# Lee la tabla de relación con las etiquetas
labels = pd.read_parquet('clean_data/IdBeneficiario_labels.parquet')
# Lee la tabla de datos nutricionales limpia
tom = pd.read_parquet('clean_data/tomas_nutricionales.parquet')
# Asigna a cada beneficiario la etiqueta correspondiente
soc = soc.merge(labels, on='IdBeneficiario', how='left')
# Lee el archivo con datos demográficos por municipio y departamento
geo = pd.read_csv('auxiliary_data/municipio_poblacion_sexo_edad_pib.csv')
geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2244 entries, 0 to 2243
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     2244 non-null   int64  
 1   DP             2244 non-null   int64  
 2   DPNOM          2244 non-null   object 
 3   DPMP           2244 non-null   int64  
 4   MPIO           2244 non-null   object 
 5   AÑO            2244 non-null   int64  
 6   Hombres_0      2244 non-null   int64  
 7   Hombres_1      2244 non-null   int64  
 8   Hombres_2      2244 non-null   int64  
 9   Hombres_3      2244 non-null   int64  
 10  Hombres_4      2244 non-null   int64  
 11  Hombres_5      2244 non-null   int64  
 12  Mujeres_0      2244 non-null   int64  
 13  Mujeres_1      2244 non-null   int64  
 14  Mujeres_2      2244 non-null   int64  
 15  Mujeres_3      2244 non-null   int64  
 16  Mujeres_4      2244 non-null   int64  
 17  Mujeres_5      2244 non-null   int64  
 18  Total Ho

### **Exploración**

In [ ]:
# Número de municipios en nuestro dataset vs el total de municipios en Colombia
soc.cod_mpio.nunique(), geo.DPMP.nunique()

(1050, 1122)

In [ ]:
# Tabla de datos con proyecciones poblacionales por municipio, sexo y edad
geo.head()

Unnamed: 0  DP      DPNOM  ...  Total Mujeres    Total           PIB
0           0   5  Antioquia  ...        1286471  2427129  55082.301180
1           1   5  Antioquia  ...        1314619  2483545  59974.806184
2           2   5  Antioquia  ...           9833    20367    231.172055
3           3   5  Antioquia  ...           9748    20258    257.196344
4           4   5  Antioquia  ...           1243     2695     28.562662

[5 rows x 22 columns]

### **Población en cada condición por municipio**

In [ ]:
# Cuenta los beneficiarios por municipio, en total y en cada condición
# (desnutrición, recuperación, reincidencia)
mpio_incidencia = soc.groupby('cod_mpio')[['IdBeneficiario', 'desnutricion', 'recuperacion', 'reincidencia']].\
  apply(lambda x: (x > 0).sum()).reset_index().rename(columns={'IdBeneficiario': 'n_beneficiarios'})
mpio_incidencia.head()

cod_mpio  n_beneficiarios  desnutricion  recuperacion  reincidencia
0      5001            59496          2903          2507           244
1      5002              885            68            60             5
2      5004              141             4             3             0
3      5021              342             2             1             0
4      5030             1115            82            68             5

In [ ]:
# Toma cada columna y la divide por la anterior para hallar:
# - de los beneficiarios en cada municipio, la fracción que sufrió desnutrición al menos una vez
# - de los beneficiarios que sufrieron desnutrición, la fracción que se recuperó
# - de los beneficiarios que se recuperaron, la fracción que reincidió
for col_index in range(3):
  col_num = mpio_incidencia.columns[col_index + 2]
  col_denom = mpio_incidencia.columns[col_index + 1]
  mpio_incidencia[f'fracc_{col_num}'] = mpio_incidencia[col_num] / mpio_incidencia[col_denom]
mpio_incidencia.head()

cod_mpio  n_beneficiarios  ...  fracc_recuperacion  fracc_reincidencia
0      5001            59496  ...            0.863589            0.097327
1      5002              885  ...            0.882353            0.083333
2      5004              141  ...            0.750000            0.000000
3      5021              342  ...            0.500000            0.000000
4      5030             1115  ...            0.829268            0.073529

[5 rows x 8 columns]

### **Cruce con los datos del Censo Nacional de Población y Vivienda 2018**

Se utiliza la población de 2018 como el estimado más general de la población por municipio, pero se tienen en cuenta los beneficiarios atendidos en los tres años (2017 a 2019).

In [ ]:
# Población y PIB por municipio
geo2018 = geo.loc[geo['AÑO'] == 2018, ['DPMP', 'MPIO', 'Total', 'PIB']]
geo2018_mpio = geo2018.groupby(['DPMP', 'MPIO'])['Total', 'PIB'].sum().reset_index()
geo2018_mpio.columns = ['cod_mpio', 'mpio', 'poblacion', 'PIB']
geo2018_mpio.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


cod_mpio        mpio  poblacion           PIB
0      5001    Medellín    2427129  55082.301180
1      5002   Abejorral      20367    231.172055
2      5004    Abriaquí       2695     28.562662
3      5021  Alejandría       4657     52.334339
4      5030       Amagá      30227    492.098704

In [ ]:
# Cruce de tablas
join = geo2018_mpio.merge(mpio_incidencia, on='cod_mpio', how='inner')
join.head(2)

cod_mpio       mpio  ...  fracc_recuperacion  fracc_reincidencia
0      5001   Medellín  ...            0.863589            0.097327
1      5002  Abejorral  ...            0.882353            0.083333

[2 rows x 11 columns]

In [ ]:
# Poblaciones en cada condición (en nuestros datos) y del PIB per cápita (por cada 100 000 individuos)
for col in ['desnutricion', 'recuperacion', 'reincidencia', 'PIB']:
  join[f'{col}_per_cap'] = join[col] / join['poblacion'] * 1e5
join.head()

cod_mpio        mpio  ...  reincidencia_per_cap  PIB_per_cap
0      5001    Medellín  ...             10.053030  2269.442670
1      5002   Abejorral  ...             24.549516  1135.032432
2      5004    Abriaquí  ...              0.000000  1059.839051
3      5021  Alejandría  ...              0.000000  1123.777945
4      5030       Amagá  ...             16.541503  1628.010400

[5 rows x 15 columns]

In [ ]:
# municipios con mayor porcentaje de reincidencia en desnutrición entre los beneficiarios recuperados
join.sort_values('fracc_reincidencia', ascending=False).head()

cod_mpio       mpio  ...  reincidencia_per_cap  PIB_per_cap
846     68370     Jordán  ...             76.103501   922.797007
305     17446  Marulanda  ...             38.358266  1192.959901
432     25095    Bituima  ...             40.535063  1818.871472
315     17665   San José  ...             62.047570  1933.450546
296     17050   Aranzazu  ...             75.779104  1223.745185

[5 rows x 15 columns]

### **Muestras con máximo 500 ó 50 beneficiarios por municipio**

In [ ]:
# Incluye variables socioeconómicas relevantes
cols_soc = ['Id', 'Estrato', 'cod_mpio', 'cod_dpto', 'ingresos_promP_imp']
cols_drop = ['desnutricion', 'desnutricion_previa',	'desnutricion_siguiente']
tom = tom.merge(soc[cols_soc], on='Id', how='inner').drop(columns=cols_drop)
tom

IdToma  Registro  Vigencia  ... cod_mpio cod_dpto ingresos_promP_imp
0               4    988319      2017  ...     5360        5           138000.0
1               5   1741422      2018  ...    11001       11           280000.0
2               6   1741422      2018  ...    11001       11           280000.0
3               7   1741422      2018  ...    11001       11           280000.0
4               8   1741422      2018  ...    11001       11           280000.0
...           ...       ...       ...  ...      ...      ...                ...
9137407  19274563   1676127      2018  ...    41530       41           133333.0
9137408  19274836   1676127      2018  ...    41530       41           133333.0
9137409  19275017   1676127      2018  ...    41530       41           133333.0
9137410  19275072   1676127      2018  ...    41530       41           133333.0
9137411  19271529   1676136      2018  ...    73319       73           108333.0

[9137412 rows x 34 columns]

In [ ]:
# Toma una muestra de máximo 500 beneficiarios por cada municipio
muestras = list()
mpios = set(soc['cod_mpio'])
for cod_mpio in mpios:
  IdBs_mpio = soc.loc[soc['cod_mpio'] == cod_mpio, 'IdBeneficiario']
  n_muestra = min(len(IdBs_mpio), 500)
  muestra_mpio = IdBs_mpio.sample(n_muestra, random_state=12)
  muestras.append(muestra_mpio)
IdB_muestra = pd.concat(muestras)
tom_max_500 = tom[tom['IdBeneficiario'].isin(IdB_muestra)] 
tom_max_500.shape

(2783633, 34)

In [ ]:
# Toma una muestra de máximo 500 beneficiarios por cada municipio
muestras = list()
mpios = set(soc['cod_mpio'])
for cod_mpio in mpios:
  IdBs_mpio = soc.loc[soc['cod_mpio'] == cod_mpio, 'IdBeneficiario']
  n_muestra = min(len(IdBs_mpio), 50)
  muestra_mpio = IdBs_mpio.sample(n_muestra, random_state=12)
  muestras.append(muestra_mpio)
IdB_muestra = pd.concat(muestras)
tom_max_50 = tom[tom['IdBeneficiario'].isin(IdB_muestra)] 
tom_max_50.shape

(328366, 34)

### **Guardado**

In [ ]:
# Guarda las tablas de datos en la carpeta de la aplicación
join.to_parquet('app/incidencia_mpio.parquet')
tom_max_500.to_parquet('app/tomas_max_500.parquet')
tom_max_50.to_parquet('app/tomas_max_50.parquet')
print(f'Archivos guardados: {time.time() - t0:.2f} s.')

Archivos guardados: 61.66 s.
